In [ ]:
# %--------------------------------------------------------------------------
# %                                 RADAR B-SCAN
# %--------------------------------------------------------------------------
# % RESEARCH PROJECT:
# % ARTIFICIAL INTELLIGENCE-BASED ALGORITHMS FOR ENHANCED DISCRIMINATION OF 
# % LANDMINES FROM GPR DATA
# %
# % UNIVERSIDAD NACIONAL DE COLOMBIA
# % TECHNOLOGY INNOVATION INSTITUTE - DIRECTED ENERGY RESEARCH CENTER
# %
# % RADAR: UWB MIMO-GPR SH-3140 ILMSENS
# % This system is composed of four ultra-wideband M-sequence units, labeled 
# % as T1R2, which are internally synchronized by the CLK unit. Each T1R2 
# % unit has one transmitting channel and two receiving channels.
# %
# % # RADAR HEAD: 4
# % # CHANNELS RX PER HEAD: 2
# % # CHANNELS TX PER HEAD: 1
# % # TOTAL RX: 8
# % # TOTAL TX: 4
# %--------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import re
import os
import h5py
import scipy.io
import matplotlib.pyplot as plt

Mounted at /content/drive


In [ ]:
def ReadDataRead(rute):
  """
  In this function is extracter data the UWBdata.m the A-scan. return Tensor the (8, frames, 511)
  """
  f = scipy.io.loadmat(rute)
  #print(f.keys()) #You look at aditional data on meansures (print title dict)   
  frames=f['numFrames'] # in this data Frames
  A=f['UWBdata00001'] # in this values the data Frame (It has all A-scan)
  B=np.zeros((8,500,511)) # define dimensions of Tensor 
  B1=A[0][1] # Data the A-scan [max100]
  for j in range(8):  # number the head 
    for k in range(int(frames/100)): # Cycle for extract the 100 data of total the frames
      B1=A[k][1]
      for i in range(100): 
        B[j][i+k*100]=[A3[i][j] for A3 in B1] # generate Tensor three dimension where extract data frame per head. 
       
  return B

In [ ]:
#this function genary in DataFrame in Pandas with rote the measures and other caracteristic than we want graphics. 
import re
def routes(route, objs, confs, days, measures, deeps, poss, metralles=[""]):

  """
   In this function, radargram processing is performed using specific parameters.

  Parámetros:
      - route (str) The file path of the measurements data. The file path should be provided as a string in the format '/path/to/file'
      - objs (list) A list of strings representing the different types of target objects to be processed. The possible values are 'IEDs' or 'Cylinder'.
      - confs (list) A list of strings representing the setup configurations for the measurements data. The possible values are 'Monostatic_1'.
      - day (list): A list of strings representing the days on which the measurements were taken. The possible values are 'Day_1'  at  ‘Day_3’
      - measure (list): A list of integers representing the measurement numbers to be processed. The possible values are 1 at 30.
      - deep (list): A list of integers representing the depth of the target object IEDs, for Cylinder is  high antenna. The possible values are 1 for 5cm and 2 for 10cm
      - pos (list): A list of integers representing the position of the target object. The possible values are 1 to 4.
      - metralle (list): A list of strings representing whether or not the IED has shrapnel. The possible values are 'M' or 'SM'.

  Return:

      - Dataframe of Pandas with descriptor of the measures. 

  """

  # Initialize empty lists for file and text
  file=[]
  text=[]

  # Loop through different parameters make patch of UWBdata
  for obj in objs:  #by now only IEDs and Cylinder
    for conf in confs:
      for day in days:
        for measure in measures:
          measur=int(measure)/100
          a=f"{measur:.2f}".replace('0.','')
          for deep in deeps:
            for pos in poss:
              for metralle in metralles:
                if obj=='IEDs':
                  ruta=f'{route}/{obj}/{conf}/{day}/IED_0{a}/IED_0{a}_D{deep}_P{pos}_{metralle}'
                elif obj=='Cylinder':
                  ruta=f'{route}/{obj}/{conf}/{day}/Cylinder_0{a}/Cylinder_0{a}_TSA800/Cylinder_0{a}_P{pos}_H{deep}_TSA800'

                # Read readme.txt file and extract relevant information using regex pattern  
                with open(f'{ruta}/readme.txt', 'r') as files:
                  readme=(files.read())
                  pattern = r'coupler-(.+?)-port'
                  wire = re.findall(pattern, readme, flags=re.DOTALL)
                  file.append([obj, day, measure, deep, pos, metralle, readme, wire, ruta])

  df=pd.DataFrame(columns=['Object','day','measure','deep','position','metralle','readme','wire','route'])
  for s in file:
    
    data = {'Object':s[0],'day':s[1],'measure':s[2],'deep':s[3],'position':s[4],'metralle':s[5],'readme':s[6],'wire':str(s[7]),'route':s[8]}
    # Agregar una nueva fila al DataFrame con los valores del diccionario
    df = df.append(data, ignore_index=True)
    # Update column values in the DataFrame
    df.loc[df['deep'] == 1, 'deep'] = "5 cm"
    df.loc[df['deep'] == 2, 'deep'] = "10 cm"
    df.loc[df['metralle'] == 'M', 'metralle'] = "with shrapnel"
    df.loc[df['metralle'] == 'SM', 'metralle'] = "without shrapnel"
    df.loc[df['Object'] == 'Cylinder', 'metralle'] = ""
    df = df.drop_duplicates()
    df['wire'] = df['wire'].str.strip()
  return df


In [ ]:
#This fuction in  grafica pure/clear raw  using the DataFrame

def graphics(df, process=['raw'], save='off', grid='off', lim=[8,1]):

  if grid=='on':
    plt.style.use('ggplot')
  else:
    plt.style.use('default')  

  """
  The function "graphics" takes a Pandas DataFrame as input, which contains the measurement data of objects.

  Parameters:
      - process (str): Represents different types of processing data that can be applied to the measurements
      - save (str): Represents the method of saving the graphics. It can take three values:
      - grid (str): Represents whether the grid should be displayed on the graphics or not
      - lim (list): Represents the limits for the y-axis of the graphics.

  Return:
      Tensor (4,500,511) (array numpy), this data processed 
  """
  x=len(df)
  vector=np.zeros((x,511,500))

  if process[0]=='all':
    process=['raw','rm_ant','rm_soil','background']

  n=len(process)
  for x in range(x):

    route= df.iloc[x]['route']

    obj = df.iloc[x]['Object']  
    day = df.iloc[x]['day']  
    measure = df.iloc[x]['measure']  
    deep = df.iloc[x]['deep']  
    pos = df.iloc[x]['position']  
    metralle = df.iloc[x]['metralle']  

    if df.iloc[x]['Object']=='IEDs':
      title = f'{obj} ({measure})    {day}   deep={deep}    pos={pos}    {metralle}'
    if df.iloc[x]['Object']=='Cylinder':
      title = f'{obj} ({measure})   {day}    High antenna = {deep}    pos={pos}'

    antenna = df.iloc[x]['wire']
    pattern = r'cable(.+?)'
    antenne = re.findall(pattern, antenna , flags=re.DOTALL)
    B = ReadDataRead(f'{route}/UWBdata.mat')
    H3_A=B[4];  # H3-RX1


    if n>1:
      fig, axs = plt.subplots(1, n, figsize=(5*n,5))
    else:
      fig, axs = plt.subplots() 

    for i, proces in enumerate(process, start=0):

      if proces=='background':

        sh = H3_A[0].argmax() #date the Shift.  
        st=20
        shift =-sh+st

        # shift the all matrice

        H3_A_sh=np.roll(H3_A,shift)

        # filter mean backgrond removal

        H3_A_mean = np.mean(H3_A_sh, axis=0)
        H3A=(H3_A_sh-H3_A_mean).T
        #vector[3]=H3_A_sh
        data=H3A
        suptitle='Background Removal'
              
      

      elif proces=='rm_ant' or proces=='rm_soil':

        if antenne[0]=='1':
          rute_ant='/content/drive/Shareddrives/TII UNAL GPR/Data Set Measurements/Dinamic Antenna/Monostatic/dynamic_wire_1'
        
        elif antenne[0]=='3':
          rute_ant='/content/drive/Shareddrives/TII UNAL GPR/Data Set Measurements/Dinamic Antenna/Monostatic/dynamic_wire_3'


        ante = ReadDataRead(f'{rute_ant}/UWBdata.mat')

        # ant=np.mean(ante[4][0:100], axis=0)
        ant = ante[4][0]
        sh = ant.argmax() #date the Shift.  
        st=20
        ant=np.roll(ant, -sh+st)  # position initial and reference

        corr_xy = np.correlate(ant, H3_A[1], mode='full') #correlate antenna vs Data Frame
        shift = corr_xy.argmax()+1 # calcule data shift

        H3_A_sh=np.roll(H3_A,shift)
        H3_A_ef=(H3_A_sh-ant)

        data=H3_A_ef.T
        vec=H3_A_sh[:][:]
        vector[x]=vec.T

        #vector[1]=data.T
        suptitle='Remove effect antenna'


      if proces=='rm_soil':

        H3_A_ef_max=H3_A_ef[0].argmax()  
        H3_A_ef_min=H3_A_ef[0].argmin()
        green=np.zeros(511)
        delta=2
        for n in range(511):
          if n>= H3_A_ef_min-2*delta:
            if n<= H3_A_ef_max+2*delta:
              green[n]=1
            else:
              green[n]=0
          else:
            green[n]=0

        syn_green=H3_A_ef*green

        #rest soil for RadarGrama


        H3_A_green=(H3_A_ef-syn_green)

        H3_A_green_mean = np.mean(H3_A_green, axis=0)
        H3A_green=(H3_A_green-H3_A_green_mean).T

        data=H3A_green
        #vector[2]=data.T
        suptitle='filter soil with Green synthetic function'

      if proces=='raw':

        sh = H3_A[0].argmax() #date the Shift.  
        st=20
        shift =-sh+st

        # shift the all matrice

        H3_A_sh=np.roll(H3_A,shift)

        # data=H3_A_sh.T
        # vec=H3_A_sh[:][:]
        # vector[x]=vec.T
        suptitle='raw Data'



    # Graphics  
      n=len(process)
      if n==1:
        plt.suptitle(suptitle, fontsize=14, fontweight='bold')
        plt.title(suptitle, fontsize=12)
        plt.imshow(data, extent=[1, 500, 40, 0], aspect='auto')
        plt.ylim(lim[0],lim[1])
        plt.xlabel('milimeter')
        plt.ylabel('Time [ns]')


      else:
        #axs[i].set_suptitle(suptitle, fontsize=14, fontweight='bold')
        axs[i].set_title(suptitle, fontsize=12)
        axs[i].imshow(data, extent=[1, 500, 40, 0], aspect='auto')
        axs[i].set_ylim(lim[0],lim[1])
        axs[i].set_xlabel('milimeter')
        axs[i].set_ylabel('Time [ns]')

    plt.suptitle(title, fontsize=14, fontweight='bold')
    rute_save = f'{route}/{process}.png'
    if save=='fig':
        # route and name of file at save.
      plt.savefig(rute_save)

    elif save=='rm':

      if os.path.isfile(rute_save):
          try:
              # Borrar el archivo
              os.remove(rute_save )
              print(f'file {rute_save} deleted.')
          except Exception as e:
              print(f'Error deleting file {rute_save}: {e} :(')
      else:
          print(f'File {rute_save} not is.')

    elif save=='data':
      arr_2d = vector.reshape(vector.shape[0], -1)
      Tensor = pd.DataFrame(arr_2d)
      Tensor.to_csv(f'{route}/UWBdata_process.csv', index=False)

    elif save=='rm_data':

      Tensor_data=f'{route}/UWBdata_process.csv'    
      if os.path.isfile(Tensor_data):
          try:
              # Borrar el archivo
              os.remove(Tensor_data )
              print(f'file {Tensor_data} deleted.')
          except Exception as e:
              print(f'Error deleting file {Tensor_data}: {e} :(')
      else:
          print(f'File {Tensor_data} not is.')      
 

    plt.show()
  return vector

In [ ]:

route ='/content/drive/Shareddrives/TII UNAL GPR/Data Set Measurements' #rute file of the measurements

obj=['IEDs']#, 'Cylinder']  # objeto the radargram    'IEDs' or 'Cylinder'
conf=['Monostatic_1'] # setup medition 
day=['Day_1']#, 'Day_2', 'Day_3'] # medition days
measure=np.arange(1,31)# number object
deep =[1,2]    # Deep the object [1 - 2] 1 for 5cm   and  2 for 10 cm 
pos = [1,2,3,4]   #Position the [1 at 4]  
metralle = ['M','SM'] # this are SM and M   "with shrapnel" or "without shrapnel".

df=routes(route, obj, conf, day, measure, deep, pos, metralle)
df
Ladmine = graphics(df, process=['raw'], save='off', grid='off', lim=[8,1])

In [ ]:
x,y,z = Ladmine.shape

mine=Ladmine[:,19:119,:]
mine.shape
mine_label=np.ones((x))
mine_label.shape

(480,)

In [ ]:
clutter = graphics(df, process=['raw'], save='off', grid='off', lim=[8,1])

In [ ]:
x,y,z = clutter.shape

clutter_cy=clutter[:,19:119,:]
clutter_cy.shape
clutter_label=np.zeros((x))
clutter_label.shape

# plt.title('hola', fontsize=12)
# plt.imshow(clutter_cy[0], extent=[1, 500, 100, 0], aspect='auto')
# plt.xlabel('milimeter')
# plt.ylabel('Time [ns]')


(120,)

In [ ]:
import h5py

print(mine.shape)
print(mine_label.shape)
print(clutter_cy.shape)
print(clutter_label.shape)

datas = np.concatenate((mine, clutter_cy), axis=0)
labels = np.concatenate((mine_label, clutter_label), axis=0)

print(datas.shape)
print(labels.shape)

print(labels)
## Crear Tensores para importar los datos. 

with h5py.File('datas.h5', 'w') as f:
    f.create_dataset('datas', data=datas)

with h5py.File('labels.h5', 'w') as f:
    f.create_dataset('labels', data=labels)


In [ ]:
# Importar los datos 
import h5py

with h5py.File('datas.h5', 'r') as f:
    data = f['datas'][:]

with h5py.File('labels.h5', 'r') as f:
    label = f['labels'][:]
